In [1]:
from newspaper import Article
import pickle as pkl
import pandas as pd
from tqdm import tqdm
import os
from pathlib import Path

from DataBase import Database
import sqlite3

In [2]:
## Importe os links gerados pelo Minerador
with open("Links/Backup_nba.pkl", "rb") as f:
    links = pkl.load(f)


links = list(set(links))

In [3]:
db = Database()
db.connect("data/Database.db")

In [ ]:
## Defina o nome do esporte para a criação na tabela no banco de dados
sport = "nba"
db.cur.execute(f'''CREATE TABLE {sport} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    data TEXT NOT NULL,
    content TEXT NOT NULL
)''')

## Conferir se a tabela foi criada
res = db.cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

In [6]:
for link in tqdm(links[2:], "Sport News"):
    article = Article(link)
    article.download()
    article.parse()

    authors = f"{article.authors}"
    data = article.publish_date
    title = article.title
    content = article.text

    sql = f'INSERT INTO {sport}(title, author, data, content) VALUES (?, ?, ?, ?)'
    db.cur.execute(sql, (
        ('title', title), 
        ('author', authors),
        ('data', data),
        ('content', content)
        ))
    db.commit()
    

    del article

Sport News:   0%|          | 0/26 [00:07<?, ?it/s]


ArticleException: Article `download()` failed with HTTPSConnectionPool(host='www.slamonline.com', port=443): Read timed out. (read timeout=7) on URL https://www.slamonline.com/news/nba/jayson-tatum-breaks-boston-record-for-most-50-point-games/

## Exemplo de uso da Biblioteca:

In [ ]:
url = "https://www.slamonline.com/news/nba/new-off-broadway-play-king-james-examines-how-lebrons-time-in-cleveland-helped-bring-two-strangers-together/"
article = Article(url)

In [ ]:
article.download()
article.parse()

In [ ]:
article.title

'“King James” Play Examines How LeBron Helped Bring Two Strangers Together'

In [ ]:
article.authors

['Brooks Warren',
 'Haben Fessehazion',
 'Drew Ruiz',
 'Deyscha Smith',
 'Slam Staff',
 'Ryan Jones',
 'Ian Pierno']

In [ ]:
article.publish_date

datetime.datetime(2023, 6, 8, 17, 5, 32, tzinfo=tzoffset(None, -14400))

In [ ]:
article.text

'“King James,” the new off-Broadway play, has taken over NYC in recent weeks. The production examines how sports fandom can bring people from different walks of life together and even spark friendships that otherwise may never be formed. In this case, it focuses on two young men in Cleveland who meet right as LeBron James is entering his rookie season with the Cleveland Cavaliers in 2003. The play proceeds to study how their friendship evolves throughout the years, and even hits a few bumps on the road, all while LeBron’s career is simultaneously evolving—the play starts in 2003 but ends over a decade later, following LeBron from Cleveland to Miami and back to Cleveland.\n\nChris Perfetti (Abbott Elementary, Gotham, Blue Bloods) plays Matt and Glenn Davis (actor, producer and Artistic Director of Steppenwolf Theatre Company; The Christians, Head of Passes, The Brother/Sister Plays, Downstate) plays Shawn. Collectively they perfectly (serious when it needs to be, and hilarious just as a